# 数据准备
# 使用dpdata这个库进行数据准备，并且划分为训练和验证集
## 下面以abacus数据为示例
## 附上deepmd和dpdata的tutorial
### https://docs.deepmodeling.com/projects/dpdata/en/master/index.html
### https://docs.deepmodeling.com/projects/deepmd/en/master/data/dpdata.html

In [12]:
! /public/software/mambaforge/envs/deepmd/bin/dp --version

DeePMD-kit v3.0.2


In [1]:
import dpdata 
import numpy as np

# 加载数据
data = dpdata.LabeledSystem('abacus_md', fmt = 'abacus/md') 
print('# the data contains %d frames' % len(data))

# 随机选择40个数据抽为验证集
index_validation = np.random.choice(201,size=40,replace=False)

# 抽取其他数据作为训练数据
index_training = list(set(range(201))-set(index_validation))
data_training = data.sub_system(index_training)
data_validation = data.sub_system(index_validation)

# 保存为训练数据
data_training.to_deepmd_npy('DeePMD-kit_Tutorial/00.data/training_data')

# 保存验证数据
data_validation.to_deepmd_npy('DeePMD-kit_Tutorial/00.data/validation_data')

print('# the training data contains %d frames' % len(data_training)) 
print('# the validation data contains %d frames' % len(data_validation)) 

# the data contains 201 frames
# the training data contains 161 frames
# the validation data contains 40 frames


## 模型参数文件

In [10]:
import json

data = {
    "_comment": " model parameters",
    "model": {
        "type_map": ["C", "O", "H"],
        "descriptor": {
            "type": "se_e2_a",
            "rcut_smth": 0.50,
            "rcut": 6.00,
            "neuron": [25, 50, 100],
            "resnet_dt": False,
            "axis_neuron": 16,
            "seed": 1,
            "_comment": " that's all"
        },
        "fitting_net": {
            "neuron": [240, 240, 240],
            "resnet_dt": True,
            "seed": 1,
            "_comment": " that's all"
        },
        "_comment": " that's all"
    },
    "learning_rate": {
        "type": "exp",
        "decay_steps": 5000,
        "start_lr": 0.001,
        "stop_lr": 3.51e-8,
        "_comment": "that's all"
    },
    "loss": {
        "type": "ener",
        "start_pref_e": 0.02,
        "limit_pref_e": 1,
        "start_pref_f": 1000,
        "limit_pref_f": 1,
        "start_pref_v": 0,
        "limit_pref_v": 0,
        "_comment": " that's all"
    },
    "training": {
        "training_data": {
            "systems": ["/public/home/huangyiru/My-Work/CACFM/aml-intro-main 2/learningnotes3/DeePMD-kit_Tutorial/00.data/training_data"],
            "batch_size": "auto",
            "_comment": "that's all"
        },
        "validation_data": {
            "systems": ["/public/home/huangyiru/My-Work/CACFM/aml-intro-main 2/learningnotes3/DeePMD-kit_Tutorial/00.data/validation_data"],
            "batch_size": 1,
            "numb_btch": 3,
            "_comment": "that's all"
        },
        "numb_steps": 1000,
        "seed": 10,
        "disp_file": "lcurve.out",
        "disp_freq": 100,
        "save_freq": 500,
        "_comment": "that's all"
    },
    "_comment": "that's all"
}

# 保存为 JSON 格式
with open("input.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

print("JSON 数据已保存为 input.json")


JSON 数据已保存为 input.json


# 训练

In [11]:
import os
os.makedirs("result", exist_ok=True)
!cd result && /public/software/mambaforge/envs/deepmd/bin/dp --tf train ../input.json


2025-05-06 16:26:58.703527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-06 16:26:58.715786: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-06 16:26:58.719529: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-06 16:26:58.729501: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-06 16:26:59.608174: W tensorflow/compiler/tf2